In [3]:
import pandas as pd

data = pd.read_csv("textual_data_an1_cleaned_final.csv")

print(data.head())

filtered_data = data.dropna(subset=['clean_tweet_2']).copy()

# Now extract both in sync
sentences = filtered_data['clean_tweet_2'].astype(str).tolist()
y = filtered_data['label'].astype(int).tolist()

# Optional: binarize the labels
y = [1 if label != 0 else 0 for label in y]

# Confirm
print("Number of sentences:", len(sentences))
print("Number of labels:", len(y))

                                   clean_tweet_2  label
0                                          nigga      4
1                                horses retarded      5
2  nigga momma youngboy spitting real shit nigga      0
3            rt xxsugvngxx ran holy nigga today       1
4                       everybody calling nigger      1
Number of sentences: 149822
Number of labels: 149822


In [4]:
from gensim.models import FastText
model = FastText(vector_size=100, window=5, min_count=1, sg=1, epochs=10)
model = FastText.load("fasttext_model.model")

word_vector = model.wv['hate']
print("Vector for 'hate':", word_vector)

similar_words = model.wv.most_similar('hate')
print("Similar words:", similar_words)

Vector for 'hate': [-2.48300359e-01  1.09273024e-01 -6.71660364e-01  2.27219760e-01
  5.62640250e-01  4.35334861e-01 -3.13606173e-01  1.45046905e-01
  8.83883834e-01 -9.38965857e-01 -5.20260274e-01 -6.16742909e-01
  1.44336507e-01  9.28647995e-01  2.25819033e-02 -2.20994428e-01
  5.68438768e-01  5.45261726e-02 -1.63975745e-01 -1.06441414e+00
  5.85953966e-02  8.31971586e-01  5.65044768e-02 -2.93443680e-01
 -6.65605485e-01 -5.01947463e-01  5.17006874e-01 -1.88412443e-02
  1.31390961e-02  5.62422037e-01  8.77174914e-01  3.13685179e-01
  2.81409472e-01 -2.45108113e-01  8.27184618e-02  1.84413284e-01
 -2.70475745e-01  7.68536210e-01 -3.24853063e-01 -6.84299786e-03
 -7.70946443e-01  4.64807063e-01  9.36575606e-02 -5.06642580e-01
 -5.20099342e-01 -3.12623739e-01 -5.40407717e-01  5.22264242e-01
  2.42474422e-01 -1.88420102e-01  2.39965487e-02 -2.40830854e-01
 -3.88183519e-02  1.61293417e-03  5.61385788e-02 -2.33369589e-01
 -1.17531925e-01 -3.83623280e-02  3.10898662e-01  1.84851035e-01
  3.99

In [5]:
import numpy as np
from gensim.models import FastText
from scipy.sparse import csr_matrix
from collections import defaultdict

# Load FastText model
model = FastText.load("fasttext_model.model")

# Sample sentences
sentences = data['clean_tweet_2']
# Ensure all sentences are strings and handle NaN values
sentences = [str(sentence) for sentence in sentences if pd.notnull(sentence)]

# Create vocabulary
vocab = list(model.wv.index_to_key)
# Create a word-to-index dictionary for faster lookups
word_to_index = defaultdict(lambda: -1, {word: i for i, word in enumerate(vocab)})

# Create embedding matrix (X) - word embeddings
embedding_dim = model.vector_size
X = np.zeros((len(vocab), embedding_dim), dtype=np.float32)
print("Shape of X:", X.shape)
for i, word in enumerate(vocab):
    X[i] = model.wv[word]

# Create hypergraph incidence matrix (H)
num_sentences = len(sentences)
H = np.zeros((len(vocab), num_sentences), dtype=np.float32)

print("Shape of H:", H.shape)

for j, sentence in enumerate(sentences):
    for word in sentence.split():
        i = word_to_index[word]
        if i != -1:
            H[i, j] = 1

# Convert to sparse matrices
X_sparse = csr_matrix(X)  # Feature matrix
H_sparse = csr_matrix(H)  # Hypergraph incidence matrix

print("Feature Matrix Shape (X):", X_sparse.shape)
print("Incidence Matrix Shape (H):", H_sparse.shape)


Shape of X: (60373, 100)
Shape of H: (60373, 149822)
Feature Matrix Shape (X): (60373, 100)
Incidence Matrix Shape (H): (60373, 149822)


In [6]:
import dhg
import torch
from sklearn.model_selection import train_test_split

vocab = list(model.wv.index_to_key)

from collections import defaultdict
word_to_index = defaultdict(lambda: -1, {word: idx for idx, word in enumerate(vocab)})

hyperedges = []
for sentence in sentences:
    word_indices = list({word_to_index[word] for word in sentence.split() if word_to_index[word] != -1})
    hyperedges.append(word_indices)

print(f"Number of hyperedges (tweets): {len(hyperedges)}")

G = dhg.Hypergraph(num_v=len(vocab), e_list=hyperedges)

import numpy as np


embedding_dim = model.vector_size


X = np.zeros((len(vocab), embedding_dim), dtype=np.float32)


for i, word in enumerate(vocab):
    X[i] = model.wv[word]

print("Feature matrix X shape:", X.shape)

num_edges = len(hyperedges)  # each tweet is a hyperedge

train_idx, test_idx = train_test_split(
    np.arange(num_edges),
    test_size=0.2,
    stratify=y,
    random_state=42
)

train_idx = torch.tensor(train_idx, dtype=torch.long)
test_idx = torch.tensor(test_idx, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)

X_tensor = torch.tensor(X, dtype=torch.float)
print(X_tensor.shape[1])
from dhg import Hypergraph
hg = Hypergraph(len(vocab), hyperedges)

/Users/admin/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of hyperedges (tweets): 149822
Feature matrix X shape: (60373, 100)
100


In [7]:
import torch.nn as nn
import torch.nn.functional as F
from dhg.models import HGNN

class MyHGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.hgnn = HGNN(in_dim, hidden_dim, out_dim)

    def forward(self, x, hg):
        return self.hgnn(x, hg)

in_dim = X_tensor.shape[1]     # embedding dimension (100 for FastText)
hidden_dim = 64
out_dim = 2                    # binary classification: hate or not

# Recreate the model structure
text_model = MyHGNN(in_dim, hidden_dim, out_dim)

# Load the saved state_dict
text_model.load_state_dict(torch.load("model_weights.pth"))

# Set the model to evaluation mode
text_model.eval()



if isinstance(y, list):
    y = torch.tensor(y, dtype=torch.long)

print("Type of hyperedges:", type(hyperedges))
print("Length of hyperedges:", len(hyperedges))

def predict_sentence(sentence, text_model, ft_model, vocab, X_tensor):
    text_model.eval()
    words = sentence.lower().split()

    node_indices = [vocab[word] for word in words if word in vocab]

    if not node_indices:
        return "No known words in sentence."


    with torch.no_grad():
        out = text_model(X_tensor, hg)  # shape: (num_nodes, out_dim)
        sentence_embed = out[node_indices].mean(dim=0)  # average over node predictions
        prediction = torch.argmax(sentence_embed).item()
        probs = torch.softmax(sentence_embed, dim=0)

    label = "Hate Speech" if prediction == 1 else "Not Hate Speech"
    confidence = probs[prediction].item()

    #return f"Prediction: {label} (confidence: {confidence:.4f})"
    return out

ft_model = FastText.load("fasttext_model.model")
vocab_dict = {word: idx for idx, word in enumerate(vocab)}

result = predict_sentence("nigga i hate tests", text_model, ft_model, vocab_dict, X_tensor)
print(result)


Type of hyperedges: <class 'list'>
Length of hyperedges: 149822
tensor([[ 2.4595e+00, -1.1357e+00],
        [ 1.0687e+00, -4.8374e-01],
        [ 9.1407e-01, -5.1038e-01],
        ...,
        [ 1.2063e-02,  1.8870e-03],
        [ 9.1603e-03, -1.3453e-03],
        [ 1.1523e-02,  4.4529e-04]])


In [ ]:
import os
# Define paths
DATA_ROOT = ''
IMG_DIR = "/Users/admin/Documents/latefusion/test_img_resized"
GT_FILE = os.path.join(DATA_ROOT, 'MMHS150K_GT.json')
TRAIN_IDS = os.path.join(DATA_ROOT, 'splits', 'train_ids.txt')
#VAL_IDS = os.path.join(DATA_ROOT, 'splits', 'val_ids.txt')
TEST_IDS = "/Users/admin/Documents/latefusion/test_ids.txt"

import json
def load_ground_truth():
    """Load the ground truth JSON file with metadata and labels."""
    with open(GT_FILE, 'r') as f:
        gt_data = json.load(f)
    return gt_data

def load_ids_from_file(file_path):
    """Load image IDs from a file."""
    with open(file_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

# Load ground truth data
gt_data = load_ground_truth()
print(f"Total number of samples: {len(gt_data)}")

test_ids = load_ids_from_file(TEST_IDS)

#print(f"Train set size: {len(train_ids)}")
#print(f"Validation set size: {len(val_ids)}")
print(f"Test set size: {len(test_ids)}")
label_distribution = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
for metadata in gt_data.values():
    # Get majority vote for each sample
    labels = metadata['labels']
    majority_label = max(set(labels), key=labels.count)
    label_distribution[majority_label] += 1

print("Original label distribution (majority vote):")
label_names = ["NotHate", "Racist", "Sexist", "Homophobe", "Religion", "OtherHate"]
for label_idx, count in label_distribution.items():
    percentage = count / len(gt_data) * 100
    print(f"{label_names[label_idx]} ({label_idx}): {count} ({percentage:.2f}%)")

def convert_to_binary(labels):
    """
    Convert the 6-class labels to binary labels.
    
    Args:
        labels: List of original labels (0-5)
        
    Returns:
        Binary label (0 for NotHate, 1 for Hate)
    """
    # For each annotator's label, convert to binary (0 remains 0, 1-5 become 1)
    binary_labels = [1 if label > 0 else 0 for label in labels]
    
    # Take majority vote for the final label
    return round(sum(binary_labels) / len(binary_labels))

# Process all data and create a binary label mapping
binary_labels = {}
for img_id, metadata in gt_data.items():
    binary_labels[img_id] = convert_to_binary(metadata['labels'])

# Count the distribution of binary labels
label_counts = {0: 0, 1: 0}
for label in binary_labels.values():
    label_counts[label] += 1

print("Binary label distribution:")
print(f"NotHate (0): {label_counts[0]} ({label_counts[0]/len(binary_labels)*100:.2f}%)")
print(f"Hate (1): {label_counts[1]} ({label_counts[1]/len(binary_labels)*100:.2f}%)")

Total number of samples: 149823
Test set size: 10000
Original label distribution (majority vote):
NotHate (0): 124003 (82.77%)
Racist (1): 12288 (8.20%)
Sexist (2): 3671 (2.45%)
Homophobe (3): 3886 (2.59%)
Religion (4): 164 (0.11%)
OtherHate (5): 5811 (3.88%)
Binary label distribution:
NotHate (0): 112858 (75.33%)
Hate (1): 36965 (24.67%)


In [9]:
# Save binary labels to a file
with open('binary_labels.json', 'w') as f:
    json.dump(binary_labels, f)

print("Binary labels saved to binary_labels.json")

def load_binary_labels(json_path='binary_labels.json'):
    """Load binary labels from a JSON file."""
    with open(json_path, 'r') as f:
        binary_labels = json.load(f)
    return binary_labels

from tensorflow.keras.utils import Sequence  # <-- Add this line

class MMHS150KSequence(Sequence):
    """Custom data generator for the MMHS150K dataset."""
    
    def __init__(self, img_dir, binary_labels, img_ids, batch_size=32, 
                 img_size=(224, 224), shuffle=True):
        """
        Initialize the data generator.
        
        Args:
            img_dir (str): Directory with all the images
            binary_labels (dict): Dictionary mapping image IDs to binary labels
            img_ids (list): List of image IDs to include in this dataset
            batch_size (int): Batch size
            img_size (tuple): Target image size (height, width)
            shuffle (bool): Whether to shuffle the data after each epoch
        """
        self.img_dir = img_dir
        self.binary_labels = binary_labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        
        # Filter to include only images that exist
        self.valid_img_ids = []
        for img_id in img_ids:
            if img_id in binary_labels and os.path.exists(os.path.join(img_dir, f"{img_id}.jpg")):
                self.valid_img_ids.append(img_id)
        
        print(f"Found {len(self.valid_img_ids)} valid images out of {len(img_ids)} IDs")
        
        # Create indices and shuffle if needed
        self.indices = np.arange(len(self.valid_img_ids))
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        """Return the number of batches per epoch."""
        return int(np.ceil(len(self.valid_img_ids) / self.batch_size))
    
    def __getitem__(self, idx):
        """Get batch at index idx."""
        # Get batch indices
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # Initialize batch arrays
        batch_x = np.zeros((len(batch_indices), self.img_size[0], self.img_size[1], 3), dtype=np.float32)
        batch_y = np.zeros(len(batch_indices), dtype=np.int32)
        batch_ids = []
        
        # Load and preprocess images
        for i, idx in enumerate(batch_indices):
            img_id = self.valid_img_ids[idx]
            img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
            
            try:
                # Load and preprocess image
                img = Image.open(img_path).convert('RGB')
                img = img.resize(self.img_size)
                img_array = img_to_array(img)
                
                # Preprocess for the appropriate model
                # For ResNet: normalize with the ImageNet statistics
                img_array = img_array / 255.0
                img_array = (img_array - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
                
                batch_x[i] = img_array
                batch_y[i] = self.binary_labels[img_id]
                batch_ids.append(img_id)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                # Use zeros for failed images
                batch_x[i] = np.zeros((self.img_size[0], self.img_size[1], 3))
                batch_y[i] = self.binary_labels[img_id]  # Use label anyway
                batch_ids.append(img_id)
        
        return batch_x, batch_y, batch_ids
    
    def on_epoch_end(self):
        """Called at the end of each epoch."""
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def get_sample(self, idx):
        """Get a single sample for visualization."""
        img_id = self.valid_img_ids[idx]
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        
        try:
            # Load image
            img = Image.open(img_path).convert('RGB')
            img = img.resize(self.img_size)
            img_array = img_to_array(img)
            
            # Normalize the image (for visualization)
            img_array = img_array / 255.0
            
            # Get label
            label = self.binary_labels[img_id]
            
            return img_array, label, img_id
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return np.zeros((self.img_size[0], self.img_size[1], 3)), self.binary_labels[img_id], img_id
        
train_ids = sorted([f.split('.')[0] for f in os.listdir(IMG_DIR) 
                    if f.lower().endswith(('.jpg', '.png', '.jpeg')) and f.split('.')[0] in test_ids])

print(f"Selected {len(train_ids)} images for training.")

import numpy as np
# Create a sample data generator with a subset of train images
sample_train_ids = train_ids[:1000]  # Use a small subset for faster processing
sample_generator = MMHS150KSequence(IMG_DIR, binary_labels, sample_train_ids, batch_size=32)

Binary labels saved to binary_labels.json
Selected 10000 images for training.
Found 1000 valid images out of 1000 IDs


In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, ResNet101, EfficientNetB0, VGG16, MobileNetV2
from tensorflow.keras.models import Model
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bar
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array


def get_feature_extractor(model_name='resnet50', img_size=(224, 224)):
    models_dict = {
        'resnet50': ResNet50,
        'resnet101': ResNet101,
        'efficientnet_b0': EfficientNetB0,
        'vgg16': VGG16,
        'mobilenet_v2': MobileNetV2
    }
    
    if model_name not in models_dict:
        raise ValueError(f"Unsupported model: {model_name}. Choose from {list(models_dict.keys())}")

    base_model = models_dict[model_name](
        weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3)
    )
    
    base_model.trainable = False  # Freeze model weights

    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    feature_extractor = Model(inputs=base_model.input, outputs=x, name=f"{model_name}_feature_extractor")

    return feature_extractor

def extract_features(data_generator, feature_extractor):
    features = []
    labels = []
    img_ids = []
    
    for i in tqdm(range(len(data_generator)), desc="Extracting features"):
        batch_imgs, batch_labels, batch_ids = data_generator[i]
        batch_features = feature_extractor.predict(batch_imgs, verbose=0)

        features.append(batch_features)
        labels.extend(batch_labels)
        img_ids.extend(batch_ids)
    
    features = np.vstack(features)  # Stack features into a single array

    return features, np.array(labels), img_ids

# Example usage
model_name = 'resnet50'
img_size = (224, 224)
feature_extractor = get_feature_extractor(model_name=model_name, img_size=img_size)
print(f"Using {model_name} as feature extractor")

# Assuming `sample_generator` is properly defined
sample_features, sample_labels, sample_img_ids = extract_features(sample_generator, feature_extractor)
print(f"Extracted features shape: {sample_features.shape}")
import torch

# Convert extracted features to PyTorch tensor
sample_features_torch = torch.tensor(sample_features, dtype=torch.float32)
sample_labels_torch = torch.tensor(sample_labels, dtype=torch.long)  # If labels are integers


Using resnet50 as feature extractor


Extracting features: 100%|██████████| 32/32 [11:37<00:00, 21.78s/it]

Extracted features shape: (1000, 2048)


In [11]:
import torch
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
import numpy as np

def build_knn_hypergraph(features, k=5):
    """
    Construct hyperedges using k-Nearest Neighbors (k-NN).
    
    Args:
        features (torch.Tensor): Tensor of shape (N, D) where N is the number of images, D is the feature dimension.
        k (int): Number of neighbors to connect per node.
    
    Returns:
        torch.Tensor: Hypergraph incidence matrix H of shape (N, E), where E is the number of hyperedges.
    """
    N = features.shape[0]
    features_np = features.cpu().numpy()  # Convert to NumPy for k-NN computation
    
    # Compute k-NN graph
    knn = NearestNeighbors(n_neighbors=k + 1, metric='cosine').fit(features_np)
    neighbors = knn.kneighbors(features_np, return_distance=False)
    
    # Create hyperedges (excluding self-loops)
    hyperedges = []
    for i in range(N):
        hyperedges.append(set(neighbors[i][1:]))  # Exclude self (first entry is the node itself)
    
    # Convert to hypergraph incidence matrix (N x E)
    num_hyperedges = len(hyperedges)
    H = torch.zeros((N, num_hyperedges), dtype=torch.float32)
    
    for e, nodes in enumerate(hyperedges):
        for node in nodes:
            H[node, e] = 1.0  # Assign weight 1 to hyperedges
    
    return H

# Example usage:
k = 5  # Number of neighbors per node
H = build_knn_hypergraph(sample_features_torch, k)
print(f"Hypergraph Incidence Matrix Shape: {H.shape}")  # Should be (N, E)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HypergraphConv
from torch_geometric.data import Data
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

class HypergraphNeuralNetwork(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.5):
        super(HypergraphNeuralNetwork, self).__init__()

        # Hypergraph convolution layers
        self.hgconv1 = HypergraphConv(in_dim, hidden_dim)
        self.hgconv2 = HypergraphConv(hidden_dim, out_dim)

        self.dropout = dropout
    
    def forward(self, x, edge_index):
        x = F.relu(self.hgconv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.hgconv2(x, edge_index)
        return x

# Convert hypergraph incidence matrix (H) to edge_index format for PyTorch Geometric
edge_index = H.nonzero().t().contiguous()

# Define HGNN model
in_dim = sample_features_torch.shape[1]  # 2048 (ResNet50 features)
hidden_dim = 1024  # Increased hidden dimension
out_dim = len(torch.unique(sample_labels_torch))  # Number of classes
img_model = HypergraphNeuralNetwork(in_dim, hidden_dim, out_dim)

# Load the saved state_dict
img_model.load_state_dict(torch.load("model_weights2.pth"))

# Set the model to evaluation mode
img_model.eval()

Hypergraph Incidence Matrix Shape: torch.Size([1000, 1000])


HypergraphNeuralNetwork(
  (hgconv1): HypergraphConv(2048, 1024)
  (hgconv2): HypergraphConv(1024, 2)
)

In [12]:
from PIL import Image
import torch
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

def extract_single_image_feature(img_path, feature_extractor):
    """
    Extract feature vector for a single test image using the CNN feature extractor.
    """
    img = Image.open(img_path).convert("RGB")  # Load and convert to RGB
    img = img.resize((224, 224))  # Resize to match model input size
    
    # Convert image to array and preprocess
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Normalize as per ResNet50
    
    # Extract feature vector
    feature_vector = feature_extractor.predict(img_array)
    return torch.tensor(feature_vector, dtype=torch.float32)  # Convert to torch tensor

# Example usage
test_img_path = "/Users/admin/Documents/latefusion/test_img_resized/1117680664667729922.jpg"
test_feature = extract_single_image_feature(test_img_path, feature_extractor)  # Extract features

# Add the test image to the existing dataset
all_features = torch.cat([sample_features_torch, test_feature], dim=0)

# Recompute the hypergraph (including the test image)
H_new = build_knn_hypergraph(all_features, k=5)

# Convert the new hypergraph into edge_index format
edge_index_new = H_new.nonzero().t().contiguous()

# Ensure model is in evaluation mode
img_model.eval()

# Get embeddings
with torch.no_grad():
    embeddings = img_model(all_features, edge_index_new)

# Extract the last embedding (corresponding to the test image)
test_embedding = embeddings[-1].detach().cpu().numpy()

print("Test Image Embedding Shape:", test_embedding.shape)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Test Image Embedding Shape: (2,)


In [13]:
text_probs = result
image_probs = test_embedding
avg_probs = (text_probs + image_probs) / 2
avg_probs = torch.softmax(avg_probs, dim=1)

print(avg_probs)

tensor([[0.8544, 0.1456],
        [0.6787, 0.3213],
        [0.6646, 0.3354],
        ...,
        [0.4942, 0.5058],
        [0.4942, 0.5058],
        [0.4943, 0.5057]])


In [14]:
alpha = 0.6  # Adjust weights as needed
wavg_probs = alpha * text_probs + (1 - alpha) * image_probs
wavg_probs = torch.softmax(wavg_probs, dim=1)

print(avg_probs)

tensor([[0.8544, 0.1456],
        [0.6787, 0.3213],
        [0.6646, 0.3354],
        ...,
        [0.4942, 0.5058],
        [0.4942, 0.5058],
        [0.4943, 0.5057]])


In [16]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np  # Ensure NumPy is imported

# Convert NumPy arrays to PyTorch tensors (if needed)
text_probs = torch.tensor(text_probs) if isinstance(text_probs, np.ndarray) else text_probs
image_probs = torch.tensor(image_probs) if isinstance(image_probs, np.ndarray) else image_probs

# Ensure both are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_probs = text_probs.to(device)
image_probs = image_probs.to(device)
# Print to debug shape mismatch
print("Text probs shape:", text_probs.shape)
print("Image probs shape:", image_probs.shape)

# Ensure both tensors have the same batch size
min_batch = min(text_probs.shape[0], image_probs.shape[0])
text_probs = text_probs[:min_batch]  
image_probs = image_probs[:min_batch]  
# Ensure both tensors are 2D
if text_probs.dim() == 1:
    text_probs = text_probs.unsqueeze(1)  # (N,) -> (N,1)
if image_probs.dim() == 1:
    image_probs = image_probs.unsqueeze(1)  # (N,) -> (N,1)

# Concatenate the tensors
fused_embedding = torch.cat((text_probs, image_probs), dim=1)

# Check the final shape
print("Fused embedding shape:", fused_embedding.shape)

# Ensure both tensors have the same number of dimensions
if text_probs.dim() == 1:  
    text_probs = text_probs.unsqueeze(1)  # Convert (N,) -> (N, 1)

if image_probs.dim() == 1:
    image_probs = image_probs.unsqueeze(1)  # Convert (N,) -> (N, 1)

# Concatenate the tensors
fused_embedding = torch.cat((text_probs, image_probs), dim=1)

# Define the fusion layer (MLP)
fusion_layer = nn.Sequential(
    nn.Linear(fused_embedding.shape[1], 128),  # Hidden layer
    nn.ReLU(),  # Non-linearity
    nn.Linear(128, 2)  # Output layer (adjust based on number of classes)
).to(device)  # Move model to correct device

# Forward pass through MLP
final_output = fusion_layer(fused_embedding)

# Convert to probabilities if required
final_probs = F.softmax(final_output, dim=1)

# Get final class predictions
final_class = torch.argmax(final_probs, dim=1)
# Assuming `y` contains your true labels
y_true = torch.tensor(y[:min_batch]).to(device)

# Compute accuracy
correct = (final_class == y_true).sum().item()
total = y_true.size(0)
accuracy = correct / total

print("Accuracy:", accuracy)


print("Final Output (logits):", final_output)
print("Final Probabilities:", final_probs)
print("Predicted Classes:", final_class)



Text probs shape: torch.Size([2, 2])
Image probs shape: torch.Size([2])
Fused embedding shape: torch.Size([2, 3])
Accuracy: 0.0
Final Output (logits): tensor([[ 0.0237, -0.2506],
        [ 0.0699, -0.1191]], grad_fn=<AddmmBackward0>)
Final Probabilities: tensor([[0.5682, 0.4318],
        [0.5471, 0.4529]], grad_fn=<SoftmaxBackward0>)
Predicted Classes: tensor([0, 0])


/var/folders/25/6x2ghpdd4dzc3df6gt3zdn2c0000gn/T/ipykernel_4741/2110401324.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_true = torch.tensor(y[:min_batch]).to(device)


In [17]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fusion_layer.parameters(), lr=0.001)

# Training loop (example for 10 epochs)
for epoch in range(10):
    fusion_layer.train()
    optimizer.zero_grad()

    outputs = fusion_layer(fused_embedding)
    loss = criterion(outputs, y_true)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.8158861398696899
Epoch 2, Loss: 0.7695637345314026
Epoch 3, Loss: 0.7252331972122192
Epoch 4, Loss: 0.6828690767288208
Epoch 5, Loss: 0.6425489187240601
Epoch 6, Loss: 0.6042916178703308
Epoch 7, Loss: 0.5680785179138184
Epoch 8, Loss: 0.5338338613510132
Epoch 9, Loss: 0.5015227198600769
Epoch 10, Loss: 0.47110018134117126


In [18]:
# Compute accuracy
correct = (final_class == y_true).sum().item()
total = y_true.size(0)
accuracy = correct / total

print("Accuracy:", accuracy)

Accuracy: 0.0


In [19]:
from collections import Counter
print(Counter(y_true.cpu().numpy()))


Counter({1: 2})


In [20]:
print("Text probs shape:", text_probs.shape)
print("Image probs shape:", image_probs.shape)
print("y_true shape:", y_true.shape)


Text probs shape: torch.Size([2, 2])
Image probs shape: torch.Size([2, 1])
y_true shape: torch.Size([2])
